In [ ]:
from pystac_client.client import Client
import geopandas as gpd
import rioxarray as riox
import xarray as xr
import json
from dask.diagnostics import ProgressBar

from rasterio.session import AWSSession
import rasterio
import boto3

In [ ]:
# use the creds created on CDSE website
with open("/data/creds_s3.json") as f:
    cred = json.load(f)

un = cred["username"]
pw = cred["password"]

# rasterio handles S3 sessions
rio_session = AWSSession(
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name="default",
    endpoint_url="eodata.dataspace.copernicus.eu")


session = boto3.session.Session()
s3 = boto3.resource(
    's3',
    endpoint_url='https://eodata.dataspace.copernicus.eu',
    aws_access_key_id=un,
    aws_secret_access_key=pw,
    region_name='default'
)

# aws_session = boto3.Session(
#     aws_access_key_id=un,
#     aws_secret_access_key=pw,
#     region_name="default",
# )

In [ ]:
# Search using STAC api
catalog = Client.open(
    "https://stac.dataspace.copernicus.eu/v1/"
)
aoi_file = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(aoi_file).geometry[0]
search = catalog.search(collections=["sentinel-1-slc"], intersects=shp)
it = next(search.items())
url = it.assets["iw1-vv"].href

In [ ]:
# open remote dataset
with rasterio.Env(session=rio_session, AWS_VIRTUAL_HOSTING=False):
    ds = riox.open_rasterio(url, chunks="auto")

In [ ]:
# download cropped array
with ProgressBar():
    ds[0, :1500].to_zarr("/data/res/test_s3_S1.zarr", mode="w")

In [ ]:
url2 = it.assets["schema-product-iw1-vh"]
bucket = s3.Bucket("eodata")

In [ ]:
# try to find xml files
files = bucket.objects.filter(Prefix="Sentinel-1/SAR/IW_SLC__1S/2025/09/30/S1C_IW_SLC__1SDV_20250930T062813_20250930T062840_004350_0089FE_0756.SAFE/")
for f in files:
    print(f)

This fails:
```bash
./sentinel1_burst_extractor_spatiotemporal.sh -p vv -s 2025-09-25 -e 2025-09-30 -x 30.19358 -y -7.2473372
```
Example from the docs also fails. Problem with string parsing.